In [ ]:
%%javascript
document.title="local"

In [ ]:
from skimage import data, transform, io
from skimage.color import rgb2gray
import pandas as pd
import matplotlib.pyplot as plt
from MILWRM.MxIF import *

In [ ]:
sample_key = pd.read_csv("/Users/cody/Dropbox (VU Basic Sciences)/fuzzy-lasagna/resources/visium_sample_key.csv", index_col=0)

sample_key.ws_adata = sample_key.ws_adata.fillna("")

---

## Define functions

In [ ]:
import napari
import numpy as np
from napari.utils.transforms import Affine

def on_transform_changed_drag(event):
    """record affine transform as ST is manipulated"""
    viewer.layers.selection.active.affine = event.value


def registration_viewer(bg_img, fg_img):
    """
    Run napari registration plugin

    Parameters
    ----------
    bg_img : np.array
        Background image to register `fg_img` to
    fg_img : np.array
        Foreground image to register to `bg_img`
    """
    viewer = napari.view_image(bg_img)  # whole-slide image to register fg_img onto
    viewer.add_image(fg_img, opacity=0.5)  # fg_img on top with transparency
    # set up interaction box for moving/scaling fg_img
    viewer.layers.selection.active.interactive = False
    viewer.overlays.interaction_box.points = viewer.layers.selection.active.extent.world
    viewer.overlays.interaction_box.show = True
    viewer.overlays.interaction_box.show_vertices = True
    viewer.overlays.interaction_box.show_handle = True
    viewer.overlays.interaction_box.allow_new_selection = False
    viewer.overlays.interaction_box.events.transform_drag.connect(on_transform_changed_drag)
    # open viewer
    napari.run()
    # return viewer object for capturing live information
    return viewer


def extract_affine(viewer, fg_img):
    """
    Extract the inverse of the applied affine transformation from napari `registration_viewer`

    Parameters
    ----------
    viewer : napari.viewer
        Napari viewer object initialized by `registration_viewer`
    fg_img : np.array
        Foreground image to register to `bg_img`

    Returns
    -------
    affine : np.array
        Affine matrix describing transformation
    scale : tuple
        Scale from affine transformation
    shape : tuple
        New shape of transformed image
    """
    # extract inverse of affine
    cut = Affine(
        rotate=viewer.layers.selection.active.affine.inverse.rotate,
        translate=viewer.layers.selection.active.affine.translate[::-1],
    )
    affine = cut.affine_matrix
    # extract scale factor from affine object
    scale = (
        viewer.layers.selection.active.affine.scale[0],
        viewer.layers.selection.active.affine.scale[1],
        1,  # for 3D image
    )
    # calculate new shape from original and scale factor
    shape = np.array(
        np.array(fg_img.shape, dtype = np.float64) * scale,
        dtype = int,
    )  # keep original aspect ratio
    return affine, scale, shape


---
# Read in two images for registration

In [ ]:
slide_id = "SG00002"
visium_id = "8899_4"

In [ ]:
sample_key.loc[sample_key.patient_name=="SG00002", ["sample_key","ws_he","ws_flip"]]

In [ ]:
%time a = img.from_npz("/Users/cody/Downloads/{}_downsample8.npz".format(slide_id))

In [ ]:
rgb = a.show(channels=["PANCK","BCATENIN","SMA"], RGB=True)

In [ ]:
ST = io.imread("/Users/cody/Dropbox (VU Basic Sciences)/fuzzy-lasagna/data/visium/datasets/8899_AS/images_alignment/V11U23-321_D1 copy.tif")

In [ ]:
plt.imshow(ST)#[::-1,::-1,:])

In [ ]:
A = a["PANCK","BCATENIN","SMA"].copy()

In [ ]:
# flip the MxIF image horizontally (if needed)
A = A[::-1,::-1,:]

In [ ]:
# flip the MxIF image horizontally (if needed)
ST = ST[::-1,::-1,:]

## Set up napari viewer

In [ ]:
import napari

In [ ]:
viewer = registration_viewer(bg_img=A, fg_img=ST)

## Extract applied affine transformation

In [ ]:
affine, scale, shape = extract_affine(viewer=viewer, fg_img=ST)

In [ ]:
affine

In [ ]:
scale

In [ ]:
shape

In [ ]:
# save affine matrix and final shape to files so we don't have to re-do the manual registration in the future
np.savetxt(
    X=affine,
    fname="/Users/cody/Downloads/{}_downsample8_{}_affine2.csv".format(slide_id,visium_id),
    delimiter=",",
)
np.savetxt(
    X=shape,
    fname="/Users/cody/Downloads/{}_downsample8_{}_shape2.csv".format(slide_id,visium_id),
    delimiter=",",
)

### Pull image out of background and export

In [ ]:
image = transform.warp(A, transform.AffineTransform(affine), output_shape=abs(shape))

In [ ]:
plt.imshow(image)